# Linear regression

In [1]:
from pyspark.ml.regression import LinearRegression

In [2]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

21/07/28 15:48:40 WARN Utils: Your hostname, GBLON1WLZ13699 resolves to a loopback address: 127.0.1.1; using 10.164.16.79 instead (on interface eth2)
21/07/28 15:48:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/07/28 15:48:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
data_path = '/home/juvid/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/sample_linear_regression_data.txt'

In [7]:
training = spark.read.format("libsvm").load(data_path)

21/07/28 15:48:49 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [8]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

Notice that features is a collection of features

In [12]:
a_feature_vector = training.head(1)[0].features

In [19]:
a_feature_vector

SparseVector(10, {0: 0.4551, 1: 0.3664, 2: -0.3826, 3: -0.4458, 4: 0.3311, 5: 0.8067, 6: -0.2624, 7: -0.4485, 8: -0.0727, 9: 0.5658})

In [18]:
a_feature_vector.size

10

In [20]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [21]:
res = lr.fit(training)

21/07/28 15:59:06 WARN Instrumentation: [7747e190] regParam is zero, which might cause numerical instability and overfitting.
21/07/28 15:59:06 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/07/28 15:59:06 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
21/07/28 15:59:06 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
21/07/28 15:59:06 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [22]:
res.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [23]:
res.intercept

0.14228558260358093

In [26]:
summary = res.summary

In [27]:
summary.pValues

[0.9927505031240562,
 0.30967074330990396,
 0.3178269194409711,
 0.003972477331573909,
 0.5201486327242175,
 0.16213017210149872,
 0.7102819001865635,
 0.5266812209137877,
 0.46256007153356316,
 0.37825808848978526,
 0.7592692146070568]

In [33]:
summary.meanSquaredError

103.28843028724194

---

Redo with a test/train split

In [34]:
all_data = spark.read.format("libsvm").load(data_path)

21/07/28 16:01:59 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [36]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [37]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                342|
|   mean|-0.6213795488815526|
| stddev| 10.570890894672331|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [38]:
test_data.describe().show()

+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|               159|
|   mean|  2.14599437507661|
| stddev|    9.511326381021|
|    min|-23.51088409032297|
|    max|26.903524792043335|
+-------+------------------+



In [39]:
res = lr.fit(train_data)

21/07/28 16:04:06 WARN Instrumentation: [b40f6157] regParam is zero, which might cause numerical instability and overfitting.


In [40]:
test_preds = res.evaluate(test_data)

In [43]:
test_preds.rootMeanSquaredError

9.926487686438854

Predict on unlabelled data

In [45]:
unlabelled_features = test_data.select('features')

In [48]:
predictions = res.transform(unlabelled_features)
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -2.1291459114382367|
|(10,[0,1,2,3,4,5,...| -0.8862321530312158|
|(10,[0,1,2,3,4,5,...|  -2.655639009307706|
|(10,[0,1,2,3,4,5,...| -1.9196123507387894|
|(10,[0,1,2,3,4,5,...| -1.1891806926080375|
|(10,[0,1,2,3,4,5,...|-0.03146160672959908|
|(10,[0,1,2,3,4,5,...|  2.3184435415633255|
|(10,[0,1,2,3,4,5,...| -0.3695935771287562|
|(10,[0,1,2,3,4,5,...| -2.6604384451534977|
|(10,[0,1,2,3,4,5,...|   -1.55571132232169|
|(10,[0,1,2,3,4,5,...|  -0.832984412508025|
|(10,[0,1,2,3,4,5,...|   2.153324408685861|
|(10,[0,1,2,3,4,5,...|  0.4869102719824333|
|(10,[0,1,2,3,4,5,...| -1.9081788549467409|
|(10,[0,1,2,3,4,5,...|  -5.559904148263277|
|(10,[0,1,2,3,4,5,...|  -6.827287507666536|
|(10,[0,1,2,3,4,5,...|  -5.219842229281323|
|(10,[0,1,2,3,4,5,...| 0.07202955377842779|
|(10,[0,1,2,3,4,5,...|   1.430239190394722|
|(10,[0,1,2,3,4,5,...|-0.1327925

---

## Another example

In [3]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

21/08/05 13:20:40 WARN Utils: Your hostname, GBLON1WLZ13699 resolves to a loopback address: 127.0.1.1; using 10.164.81.22 instead (on interface eth2)
21/08/05 13:20:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/05 13:20:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [11]:
from pathlib import Path
data_path = Path('../Python-and-Spark-for-Big-Data-master/Course_Notes/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv')


In [13]:
df = spark.read.csv(str(data_path), header=True, inferSchema=True)

In [16]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [18]:
for item in df.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [19]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [20]:
df.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [21]:
assembler = VectorAssembler(
    inputCols=[
        'Avg Session Length',
        'Time on App',
        'Time on Website',
        'Length of Membership',
    ], 
    outputCol='features'
)

In [22]:
output = assembler.transform(df)

In [23]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



creates a new features column to the original df

In [27]:
output.head(1)[0]['features']

DenseVector([34.4973, 12.6557, 39.5777, 4.0826])

This is the format spark expects for most ML algorithms

In [28]:
final_data = output.select('features', 'Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [29]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [30]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                354|
|   mean|  495.8642468312903|
| stddev|  77.19405443156823|
|    min|   266.086340948469|
|    max|  744.2218671047146|
+-------+-------------------+



In [32]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                146|
|   mean|  507.6786010343743|
| stddev|  83.92110305950304|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [33]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [34]:
lr_model = lr.fit(train_data)

21/08/05 13:35:09 WARN Instrumentation: [3051732a] regParam is zero, which might cause numerical instability and overfitting.
21/08/05 13:35:10 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/08/05 13:35:10 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
21/08/05 13:35:10 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
21/08/05 13:35:10 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [35]:
test_results = lr_model.evaluate(test_data)

In [40]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -5.547012990786641|
| -5.253012189375966|
|  -7.09819961319937|
|-1.7627221150124797|
| 2.6516966475240906|
| 1.1391350666010567|
| -19.49748586723672|
| -7.579079422936843|
|-19.717094043756845|
|-2.1879149393911916|
| -8.538867508857663|
|-1.7570922115878034|
|  4.512023029544139|
|  8.737357406491242|
| 19.063114915486494|
| 3.3556842649929877|
|  5.137697117624441|
| -6.919305943345762|
|  8.523691080964397|
| -3.861307570878921|
+-------------------+
only showing top 20 rows



In [41]:
test_results.r2

0.9862918292946022

In [42]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [43]:
unlabelled_data = test_data.select('features')

In [44]:
prediction = lr_model.transform(unlabelled_data)

In [45]:
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8364326747734...|473.04891341777625|
|[30.8794843441274...|495.45961217423064|
|[31.0613251567161...|  494.653657671101|
|[31.2606468698795...|423.08935337196385|
|[31.3091926408918...|430.06902119240954|
|[31.3895854806643...|408.93047599338183|
|[31.5702008293202...| 565.4429780086416|
|[31.7207699002873...| 546.3540129009598|
|[31.8164283341993...| 520.8395855474132|
|[31.8186165667690...|448.60658830952684|
|[31.8854062999117...| 398.6421404813332|
|[31.9120759292006...| 389.2918085172955|
|[31.9480174211613...|457.40885386335367|
|[31.9549038566348...|431.26052253343573|
|[32.0180740106320...| 338.7199958298288|
|[32.0542618511847...|   558.51897340399|
|[32.0789475795693...| 352.7260215207673|
|[32.0883806304482...| 519.0851723315038|
|[32.1151190660142...|341.53450908288073|
|[32.1223647957977...| 532.7949261358992|
+--------------------+------------